***
## Модификаторы сравнения

В Django ORM модификаторы выполняют роль операторов, используемых в SQL для фильтрации выборки по значениям полей. Вот как переводятся самые востребованные модификаторы (перед модификатором обязательно ставится двойной символ подчёркивания):


Значение оператора	    SQL	                ORM
Равно	                `=`	                `__exact`
Сравнение с NULL	    `IS NULL`	        `__exact=None`
Больше	                `>`	                `__gt`
Больше или равно	    `>=`	            `__gte`
Меньше	                `<`	                `__lt`
Меньше или равно	    `<=`	            `__lte`
Поиск по тексту	        `LIKE '%фраза%'`    `__contains='фраза'`
Вхождение в множество	`IN (1, 3, 4)`	    `__in=[1, 3, 4]`
Вхождение в диапазон	`BETWEEN 1 AND 4`	`__range=[1, 4]`

Полный список модификаторов есть в [документации Django](https://docs.djangoproject.com/en/3.2/ref/models/querysets/#field-lookups).

***
## Метод filter()

Метод `.filter()` **принимает** на вход именованные (keyword) аргументы: 

* название поля,

* модификатор поиска,

* значение для фильтрации;

Метод `.filter()` **возвращает** QuerySet с объектами, которые соответствуют заданным условиям.

```py
<Модель>.objects.filter(<свойство>__<модификатор>=<значение для фильтрации>) 
```

Например, такой SQL-запрос… 

```sql
SELECT *
FROM ice_cream_icecream
WHERE title LIKE '%эскимо%'; 
```

…в Django ORM записывается так: 

```py
IceCream.objects.filter(title__contains='эскимо')
```  

> Самый распространённый модификатор, применяемый в методе `filter()` — `__exact`, «точное совпадение». Если в выражении сравнения модификатор не указан явно — то по умолчанию применяется `__exact`.

In [ ]:
# homepage/views.py
from django.shortcuts import render

from ice_cream.models import IceCream

def index(request):
    template_name = 'homepage/index.html'
    # Заключаем вызов методов в скобки
    # (это стандартный способ переноса длинных строк в Python);
    # каждый вызов пишем с новой строки, так проще читать код:
    ice_cream_list = IceCream.objects.values(
            'id', 'title', 'description'
        # Верни только те объекты, у которых в поле is_on_main указано True:
        ).filter(is_on_main=True)
    context = {
        'ice_cream_list': ice_cream_list,
    }
    return render(request, template_name, context)

***
## Метод exclude()

Для исключения объектов, соответствующих определённому условию, применяют метод  `.exclude()`; в SQL ему соответствует комбинация операторов `WHERE NOT ...`:

```py
<Модель>.objects.exclude(<свойство>__<модификатор>=<значение для фильтрации>) 
```

В модели `IceCream` предусмотрен флаг `is_published` — показывать ли запись на сайте. Если в поле `is_published` установлено значение `False` — эту запись нужно исключить из выборки, не показывать её ни на одной странице проекта, в том числе и на главной. 

Через метод `.exclude()` настроим запрос так, чтобы в QuerySet **не попали** те записи, у которых в поле `is_published` стоит `False`:

```py
...

def index(request):
    template_name = 'homepage/index.html'
    ice_cream_list = IceCream.objects.values(
            'id', 'title', 'description'
        # Исключи те объекты, у которых is_published=False:
        ).exclude(is_published=False)
    context = {
        'ice_cream_list': ice_cream_list,
    }
    return render(request, template_name, context)
```

***
## Работа с датами

Для работы с датами применяются специальные модификаторы. 

Предположим, в каком-нибудь проекте блога есть модель `Pоst` (публикация) и одно из полей этой модели — дата публикации.

```py
class Post(models.Model):
    pub_date = models.DateTimeField('Дата публикации')
    ... 
```

Востребованная задача для таких проектов — получить подборку записей, опубликованных за определённый диапазон времени. Для решения этой задачи в Django ORM можно применить модификатор **range** — проверить значение даты на вхождения в диапазон дат.

Получим публикации, созданные в диапазоне с 1 января 1890 до 31 марта 1895 (нашему блогу почти полтора века!):

```py
import datetime

...

def filtered_date(request):
    ...    
    start_date = datetime.date(1890, 1, 1)
    end_date = datetime.date(1895, 3, 31)
    Post.objects.filter(pub_date__range=(start_date, end_date))
    # SQL-версия запроса: WHERE pub_date BETWEEN '1890-01-01' AND '1895-03-31';
    return render(request, template_name, context)
```

Можно проверять значение поля только по части даты. Например, можно отфильтровать только те записи, которые опубликованы

* 13 числа (любого месяца и года),

* в мае (в любое число и в любой год),

* в пятницу,

* …в какие-то ещё дни или диапазоны дат.

Для таких запросов в Django ORM применяют дополнительные суффиксы `__date`, `__year`, `__month`, `__day`, `__week`, `__week_day`, `__quarter`:

```py
# Условия для конкретной даты:
Post.objects.filter(pub_date__date=datetime.date(1890, 1, 1))
# Ранее первого января 1895 года:
Post.objects.filter(pub_date__date__lt=datetime.date(1895, 1, 1))
# В конкретный год:
Post.objects.filter(pub_date__year=1890)
# В любой год с января по июнь включительно:
Post.objects.filter(pub_date__month__lte=6)
# В первый квартал любого года:
Post.objects.filter(pub_date__quarter=1) 
```